<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1608275719214-edfbc5c2-063a-4f71-adc2-67290ca8f417.png"/>

SBERT论文：[Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks](https://arxiv.org/abs/1908.10084)  
Pytorch版本Sentence-BERT仓库：https://github.com/UKPLab/sentence-transformers  
- Sentence-BERT 在BERT的最后一层上添加了一个池化操作，并经过微调，可以推导出固定大小的sentence embedding。
- SBERT 进一步使用Siamese和triplet神经网络更新模型的权重参数。
- SBERT 使用Siamese孪生网络生成包含丰富语义的sentence embeddings，用于比较cosine-similarity。

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1606268569362-33d8eac5-17b0-4df6-b891-d4d07851642c.png"/>

- $CosineSim(e_1,e_2)=\frac{e_1\cdot e_2}{\lVert e_1 \rVert \cdot \lVert e_2 \rVert}$

- SBERT采用计算两段文本向量的余弦相似度来完成预测（与模型训练过程不同），大幅缩短了响应时长。
- 例如，SBERT论文中举例：从10000个句子中找到最相似的一对问句，因可能的组合太多（$10000 * (10000-1)/2=49995000$次计算），直接使用BERT计算需要65小时，而SBERT完成推理只需要5秒。

代码成功从天池实验室点击编辑按钮加载到DSW，加载好的代码会⾃动打开，默认在<b>download</b>
⽬录下<br>
1、点击左侧的【天池】按钮<br>
2、会出现【保存到天池】按钮和【添加数据源】模块，搜索Pytorch5.4，点击数据集中的下载按钮即可<br>
###### （具体如下图所示）
<center><img
src="https://img.alicdn.com/imgextra/i4/O1CN01zsetgx1zaOBQbSDLs_!!6000000006730-2-
tps-616-589.png" width=60%></center>
核⼼问题2
数据集下载成功后，⻚⾯右上⻆会提示数据集下载成功，也会说名数据集存储位置，默认在
<b>download</b>⽬录下，如下图所示。
<center>
<img
src="https://img.alicdn.com/imgextra/i3/O1CN01uJzjgf1MLwg6jK7za_!!6000000001419-2-tps-
1409-377.png" width=60%>
<img
src="https://img.alicdn.com/imgextra/i1/O1CN01XQmAP027k1R811xls_!!6000000007834-2-
tps-857-465.png" width=60%>
</center>

# 加载相关库

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util, models
import torch
from torch import nn
import pickle
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

# 加载预训练模型

- 对于句子/文本嵌入，我们希望将可变长度的输入文本映射到固定大小的Dense Vector。
- 我们可以使用预先训练的SentenceTransformer模型，或者利用transformers上的model来从头开始创建网络体系结构。
- 我们可以使用的最基本的网络体系结构如下：

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1606268067356-f5d1ec76-0ac2-4fac-8106-3318fccd3a13.png"/>

In [ ]:
# 加载transformers上预训练模型 #
model_name = './weights/chinese_roberta_wwm_ext'
word_embedding_model = models.Transformer(model_name, max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(),
                           out_features=256, activation_function=nn.Tanh())

embedder = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
# 保证每次加载的embedding一致
if not os.path.exists('model.pth'):
    # 保存模型参数
    torch.save(embedder.state_dict(), 'model.pth')
else:
    # 加载模型参数
    embedder.load_state_dict(torch.load('model.pth'))

# 存储embedding

In [ ]:
# 存储语料embedding，方便快速取用 #
batch_size = 128
max_corpus_size = 307128  # 句子数量
dataset_path = "./totalnews.csv" 
embedding_cache_path = 'quora-embeddings-{}-size-{}.pkl'.format(model_name.replace('/', '_'), max_corpus_size)  # 词向量存储地址
# 检查 embedding cache path 是否存在
if not os.path.exists(embedding_cache_path):
    # 将csv文件中的数据转化成list
    df = pd.read_csv(dataset_path)
    corpus_sentences = list(df['title'].map(lambda x: str(x)))
    if len(corpus_sentences) >= max_corpus_size:
        corpus_sentences = corpus_sentences[:max_corpus_size]
    print("Encode the corpus. This might take a while")
    corpus_embeddings = embedder.encode(corpus_sentences, batch_size=batch_size,
                                        show_progress_bar=True, convert_to_tensor=True)

    print("Store file on disc")
    with open(embedding_cache_path, "wb") as fOut:
        pickle.dump({'sentences': corpus_sentences, 'embeddings': corpus_embeddings}, fOut)
else:
    print("Load pre-computed embeddings from disc")
    with open(embedding_cache_path, "rb") as fIn:
        cache_data = pickle.load(fIn)
        corpus_sentences = cache_data['sentences']
        corpus_embeddings = cache_data['embeddings']

print("Corpus loaded with {} sentences / embeddings".format(len(corpus_sentences)))

# 执行主函数
- 对于小型语料库（最多约10万个条目），我们可以计算查询与语料库中所有条目之间的余弦相似度。
- 对于大型语料库，对所有分数进行排序将花费太多时间。因此，我们使用util.semantic_search函数仅获取前k个条目。
    - sentence_transformers.util.semantic_search(query_embeddings: torch.Tensor, corpus_embeddings: torch.Tensor, query_chunk_size: int=100, corpus_chunk_size: int=100000, top_k: int=10)
        - 这个函数在一堆 query embeddings 和 corpus embeddings 内执行余弦相似度搜索。
        - 它可用于最多可达100万条的信息检索/语义搜索的语料库。
        - 参数：
            - query_embeddings - query embeddings的二维张量
            - corpus_embeddings - corpus embeddings的二维张量
            - query_chunk_size - 同时处理100个查询。增加这个值可以提高速度，但是需要更多的内存。
            - corpus_chunk_size - 一次扫描10万条语料。增加这个值可以提高速度，但是需要更多的内存。
            - top_k - 检索顶部k个匹配语料。注意：如果你的语料长度大于query_chunk_size，会返回。
        - 返回：
            - 返回一个按余弦相似度得分递减的排序列表。

In [ ]:
inp_question = '京津冀秋冬'

start_time = time.time()
question_embedding = embedder.encode(inp_question, batch_size=batch_size, convert_to_tensor=True)
hits = util.semantic_search(question_embedding, corpus_embeddings,
                            query_chunk_size=1, corpus_chunk_size=len(corpus_sentences),
                            top_k=10)
end_time = time.time()  # 0.16s
hits = hits[0]  # 得到第一个问题的hits

print("Input question: ", inp_question)
print("Results (after {:.3f} seconds):".format(end_time-start_time))
for hit in hits[0:5]:
    print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))

print("\n\n========\n")